In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 29.2 MB/s 
     |████████████████████████████████| 6.6 MB 45.9 MB/s 
     |████████████████████████████████| 101 kB 13.4 MB/s 
     |████████████████████████████████| 596 kB 29.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.7 MB/s 
     |████████████████████████████████| 6.4 MB 51.0 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/JU Project/ExecutiveTweetsBTC.csv')
df

,Date,Tweet
0,2020-12-08 03:55:06+00:00,Hear me out... a SPAC... ticker BTC... The Bit...
1,2021-04-13 14:38:31+00:00,No replacement for $BTC. If $RIOT were to hav...
2,2018-04-30 17:53:27+00:00,$SQ Short term tgt $30 started as innovative ...
3,2021-04-17 19:05:14+00:00,I mean. We all knew $COIN was gonna come out a...
4,2021-12-04 22:01:47+00:00,Just saw someone touting that the ETH/BTC pair...
...,...,...
298,2019-08-05 21:07:16+00:00,i cant believe am hearing @CNBC expert say 'u ...
299,2021-05-19 11:49:59+00:00,Are you buying or selling Bitcoin Today? #BTC 🪙
300,2021-12-02 19:59:57+00:00,Will gold outperform Bitcoin in 2022? $GLD $BTC
301,2021-06-13 18:33:07+00:00,Musk denies speculation Tesla has sold more bi...


In [ ]:
df['Date'] = pd.to_datetime(df['Date']).dt.date
df

,Date,Tweet
0,2020-12-08,Hear me out... a SPAC... ticker BTC... The Bit...
1,2021-04-13,No replacement for $BTC. If $RIOT were to hav...
2,2018-04-30,$SQ Short term tgt $30 started as innovative ...
3,2021-04-17,I mean. We all knew $COIN was gonna come out a...
4,2021-12-04,Just saw someone touting that the ETH/BTC pair...
...,...,...
298,2019-08-05,i cant believe am hearing @CNBC expert say 'u ...
299,2021-05-19,Are you buying or selling Bitcoin Today? #BTC 🪙
300,2021-12-02,Will gold outperform Bitcoin in 2022? $GLD $BTC
301,2021-06-13,Musk denies speculation Tesla has sold more bi...


In [ ]:
def tweets_per_day(df):
    df['Date'] = pd.to_datetime(df['Date'])
#   return df[['Tweets']].groupby(df['Date'].dt.date).count()
    tweet = df['Tweet'].groupby(df['Date'].dt.date).count() 
    d1 = pd.DataFrame(tweet)
    return d1
  # if you want output to be `Series`

df1 = tweets_per_day(df)
df1 = df1.reset_index(level=0)

In [ ]:
df1

,Date,Tweet
0,2017-03-01,1
1,2017-03-09,1
2,2017-04-26,2
3,2017-05-31,1
4,2017-07-05,1
...,...,...
240,2022-05-02,1
241,2022-05-03,1
242,2022-05-09,3
243,2022-05-10,1


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

In [ ]:
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
import math

stop_words = stopwords.words('english')
stop_words.remove("not")
stemmer = SnowballStemmer('english')

text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

def preprocess(text, stem=False):
  text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
  tokens = []
  for token in text.split():
    if token not in stop_words:
      if stem:
        tokens.append(stemmer.stem(token))
      else:
        tokens.append(token)
  return " ".join(tokens)

df.Tweets = df.Tweet.apply(lambda x: preprocess(x))
df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


,Date,Tweet
0,2020-12-08,Hear me out... a SPAC... ticker BTC... The Bit...
1,2021-04-13,No replacement for $BTC. If $RIOT were to hav...
2,2018-04-30,$SQ Short term tgt $30 started as innovative ...
3,2021-04-17,I mean. We all knew $COIN was gonna come out a...
4,2021-12-04,Just saw someone touting that the ETH/BTC pair...
...,...,...
298,2019-08-05,i cant believe am hearing @CNBC expert say 'u ...
299,2021-05-19,Are you buying or selling Bitcoin Today? #BTC 🪙
300,2021-12-02,Will gold outperform Bitcoin in 2022? $GLD $BTC
301,2021-06-13,Musk denies speculation Tesla has sold more bi...


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [ ]:
neg = []
neu = []
pos = []
i = 0
def normalize(score, alpha=15):
    """
    Normalize the score to be between -1 and 1 using an alpha that
    approximates the max expected value
    """
    norm_score = score/math.sqrt((score*score) + alpha)
    return norm_score

for tweet in df.Tweet:
  i = i+1
  encoded_tweet = tokenizer(tweet,padding=True, truncation=True,max_length=250, add_special_tokens = True, return_tensors='pt')
  output = model(**encoded_tweet)
  scores = output[0][0].detach().numpy()
  normScores = softmax(scores)  
  neg.append(normScores[0])
  neu.append(normScores[1])
  pos.append(normScores[2])
  print(i,"--> ",normScores)

1 -->  [0.0717213  0.01218547 0.9160932 ]
2 -->  [0.08107361 0.02286281 0.8960636 ]
3 -->  [0.04563661 0.73194224 0.22242114]
4 -->  [0.06549419 0.04032139 0.8941846 ]
5 -->  [0.06853938 0.04934912 0.88211155]
6 -->  [0.02389283 0.39093027 0.5851769 ]
7 -->  [0.9309407  0.01371996 0.05533948]
8 -->  [0.06852176 0.0132599  0.91821843]
9 -->  [0.89046264 0.00820894 0.10132856]
10 -->  [0.8965156  0.00993939 0.09354499]
11 -->  [0.32901067 0.05560081 0.6153886 ]
12 -->  [0.08980023 0.0226895  0.88751024]
13 -->  [0.03392616 0.02056026 0.94551367]
14 -->  [0.05082171 0.02018774 0.9289904 ]
15 -->  [0.01910692 0.4507189  0.53017414]
16 -->  [0.01794906 0.5212877  0.46076334]
17 -->  [0.01765916 0.8217745  0.1605663 ]
18 -->  [0.06071047 0.01835815 0.9209313 ]
19 -->  [0.06365345 0.2876488  0.64869773]
20 -->  [0.09653256 0.01169752 0.89176977]
21 -->  [0.02118478 0.7172566  0.2615585 ]
22 -->  [0.02647699 0.04598452 0.92753845]
23 -->  [0.0696829  0.19830203 0.73201513]
24 -->  [0.0796947  

In [ ]:
df['negative'] = neg
df['neutral'] = neu
df['positive'] = pos
df

,Date,Tweet,negative,neutral,positive
0,2020-12-08,Hear me out... a SPAC... ticker BTC... The Bit...,0.071721,0.012185,0.916093
1,2021-04-13,No replacement for $BTC. If $RIOT were to hav...,0.081074,0.022863,0.896064
2,2018-04-30,$SQ Short term tgt $30 started as innovative ...,0.045637,0.731942,0.222421
3,2021-04-17,I mean. We all knew $COIN was gonna come out a...,0.065494,0.040321,0.894185
4,2021-12-04,Just saw someone touting that the ETH/BTC pair...,0.068539,0.049349,0.882112
...,...,...,...,...,...
298,2019-08-05,i cant believe am hearing @CNBC expert say 'u ...,0.081146,0.016366,0.902488
299,2021-05-19,Are you buying or selling Bitcoin Today? #BTC 🪙,0.033131,0.040664,0.926205
300,2021-12-02,Will gold outperform Bitcoin in 2022? $GLD $BTC,0.459753,0.057765,0.482482
301,2021-06-13,Musk denies speculation Tesla has sold more bi...,0.147709,0.049229,0.803062


In [ ]:
dfx = pd.DataFrame()
po = []
nu = []
ne = []
for i in range (0,245):

  d = df['Date'][i]

  x = df[df['Date'] == d]
  po.append(x['positive'].mean())
  nu.append(x['neutral'].mean())
  ne.append(x['negative'].mean())
dfx['Date'] = df1['Date']
dfx['positive'] = po
dfx['negative'] = ne
dfx['neutral'] = nu
dfx

,Date,positive,negative,neutral
0,2017-03-01,0.916093,0.071721,0.012185
1,2017-03-09,0.896064,0.081074,0.022863
2,2017-04-26,0.222421,0.045637,0.731942
3,2017-05-31,0.894185,0.065494,0.040321
4,2017-07-05,0.895871,0.072890,0.031239
...,...,...,...,...
240,2022-05-02,0.177683,0.016686,0.805631
241,2022-05-03,0.923746,0.053084,0.023169
242,2022-05-09,0.905618,0.076749,0.017633
243,2022-05-10,0.630447,0.038508,0.331045


In [ ]:
# Set date as Index
dfx = dfx.set_index('Date')
dfx = dfx.sort_index()
dfx

,positive,negative,neutral
Date,,,
2017-03-01,0.916093,0.071721,0.012185
2017-03-09,0.896064,0.081074,0.022863
2017-04-26,0.222421,0.045637,0.731942
2017-05-31,0.894185,0.065494,0.040321
2017-07-05,0.895871,0.072890,0.031239
...,...,...,...
2022-05-02,0.177683,0.016686,0.805631
2022-05-03,0.923746,0.053084,0.023169
2022-05-09,0.905618,0.076749,0.017633


### STOCK DATA

In [ ]:
from pandas_datareader import data as pdr
import yfinance as yf

yf.pdr_override()

stock_price = pdr.get_data_yahoo("BTC-USD", start = '2017-01-01', end = '2022-05-07')
print(len(stock_price))
stock_price

[*********************100%***********************]  1 of 1 completed
1953


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-01-01,963.658020,1003.080017,958.698975,998.325012,998.325012,147775008
2017-01-02,998.617004,1031.390015,996.702026,1021.750000,1021.750000,222184992
2017-01-03,1021.599976,1044.079956,1021.599976,1043.839966,1043.839966,185168000
2017-01-04,1044.400024,1159.420044,1044.400024,1154.729980,1154.729980,344945984
2017-01-05,1156.729980,1191.099976,910.416992,1013.380005,1013.380005,510199008
...,...,...,...,...,...,...
2022-05-03,38528.109375,38629.996094,37585.621094,37750.453125,37750.453125,27326943244
2022-05-04,37748.011719,39902.949219,37732.058594,39698.371094,39698.371094,36754404490
2022-05-05,39695.746094,39789.281250,35856.515625,36575.140625,36575.140625,43106256317


In [ ]:
stock_price = stock_price.drop_duplicates()

# dropping the zero values from volume
stock_price = stock_price[stock_price['Volume'] != 0]

# filtering the important columns required
stock_price = stock_price.filter(['Date', 'Close', 'Open', 'High', 'Low', 'Volume'])

# sorting the data according to the index i.e 'Date'
stock_price = stock_price.sort_index(ascending=True, axis=0)
stock_price

,Close,Open,High,Low,Volume
Date,,,,,
2017-01-01,998.325012,963.658020,1003.080017,958.698975,147775008
2017-01-02,1021.750000,998.617004,1031.390015,996.702026,222184992
2017-01-03,1043.839966,1021.599976,1044.079956,1021.599976,185168000
2017-01-04,1154.729980,1044.400024,1159.420044,1044.400024,344945984
2017-01-05,1013.380005,1156.729980,1191.099976,910.416992,510199008
...,...,...,...,...,...
2022-05-03,37750.453125,38528.109375,38629.996094,37585.621094,27326943244
2022-05-04,39698.371094,37748.011719,39902.949219,37732.058594,36754404490
2022-05-05,36575.140625,39695.746094,39789.281250,35856.515625,43106256317


### COMBINE DATA

In [ ]:
stock_data = pd.merge(stock_price,dfx, how='left', left_index=True, right_index=True)

stock_data

,Close,Open,High,Low,Volume,positive,negative,neutral
Date,,,,,,,,
2017-01-01,998.325012,963.658020,1003.080017,958.698975,147775008,NaN,NaN,NaN
2017-01-02,1021.750000,998.617004,1031.390015,996.702026,222184992,NaN,NaN,NaN
2017-01-03,1043.839966,1021.599976,1044.079956,1021.599976,185168000,NaN,NaN,NaN
2017-01-04,1154.729980,1044.400024,1159.420044,1044.400024,344945984,NaN,NaN,NaN
2017-01-05,1013.380005,1156.729980,1191.099976,910.416992,510199008,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-05-03,37750.453125,38528.109375,38629.996094,37585.621094,27326943244,0.923746,0.053084,0.023169
2022-05-04,39698.371094,37748.011719,39902.949219,37732.058594,36754404490,NaN,NaN,NaN
2022-05-05,36575.140625,39695.746094,39789.281250,35856.515625,43106256317,NaN,NaN,NaN


In [ ]:
stock_data.sort_index(axis = 1)
stock_data

,Close,Open,High,Low,Volume,positive,negative,neutral
Date,,,,,,,,
2017-01-01,998.325012,963.658020,1003.080017,958.698975,147775008,NaN,NaN,NaN
2017-01-02,1021.750000,998.617004,1031.390015,996.702026,222184992,NaN,NaN,NaN
2017-01-03,1043.839966,1021.599976,1044.079956,1021.599976,185168000,NaN,NaN,NaN
2017-01-04,1154.729980,1044.400024,1159.420044,1044.400024,344945984,NaN,NaN,NaN
2017-01-05,1013.380005,1156.729980,1191.099976,910.416992,510199008,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-05-03,37750.453125,38528.109375,38629.996094,37585.621094,27326943244,0.923746,0.053084,0.023169
2022-05-04,39698.371094,37748.011719,39902.949219,37732.058594,36754404490,NaN,NaN,NaN
2022-05-05,36575.140625,39695.746094,39789.281250,35856.515625,43106256317,NaN,NaN,NaN


In [ ]:
stock_data.isna().any()

Close       False
Open        False
High        False
Low         False
Volume      False
positive     True
negative     True
neutral      True
dtype: bool

In [ ]:
# rearranging the columns of the whole stock_data
stock_data = stock_data[['Close', 'negative', 'neutral', 'positive', 'Open', 'High', 'Low', 'Volume']]

# displaying the final stock_data
stock_data

,Close,negative,neutral,positive,Open,High,Low,Volume
Date,,,,,,,,
2017-01-01,998.325012,NaN,NaN,NaN,963.658020,1003.080017,958.698975,147775008
2017-01-02,1021.750000,NaN,NaN,NaN,998.617004,1031.390015,996.702026,222184992
2017-01-03,1043.839966,NaN,NaN,NaN,1021.599976,1044.079956,1021.599976,185168000
2017-01-04,1154.729980,NaN,NaN,NaN,1044.400024,1159.420044,1044.400024,344945984
2017-01-05,1013.380005,NaN,NaN,NaN,1156.729980,1191.099976,910.416992,510199008
...,...,...,...,...,...,...,...,...
2022-05-03,37750.453125,0.053084,0.023169,0.923746,38528.109375,38629.996094,37585.621094,27326943244
2022-05-04,39698.371094,NaN,NaN,NaN,37748.011719,39902.949219,37732.058594,36754404490
2022-05-05,36575.140625,NaN,NaN,NaN,39695.746094,39789.281250,35856.515625,43106256317


In [ ]:
stock_data.to_csv('/content/drive/MyDrive/JU Project/EXEC_FINBERT_BTC_data.csv')